## Changing the current working Directory

In [48]:
%cd /content/drive/MyDrive/PCB_Defect_Detection

/content/drive/MyDrive/PCB_Defect_Detection


In [2]:
!pwd

/content/drive/MyDrive/PCB_Defect_Detection


In [33]:
base_path = r'/content/drive/MyDrive/PCB_Defect_Detection/'

# Cloning YOLOv5 repo

In [2]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

Cloning into 'yolov5'...
remote: Enumerating objects: 14080, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 14080 (delta 66), reused 86 (delta 60), pack-reused 13982
Receiving objects: 100% (14080/14080), 13.24 MiB | 7.42 MiB/s, done.
Resolving deltas: 100% (9720/9720), done.
/content/drive/MyDrive/PCB_Defect_Detection/yolov5
HEAD is now at fbe67e4 Fix `OMP_NUM_THREADS=1` for macOS (#8624)


# Install dependencies as necessary

In [3]:
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 1.6 MB 48.0 MB/s 
Setup complete. Using torch 1.12.1+cu113 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


# Let's look at our training environment and GPU provided

In [4]:
import torch
from IPython.display import Image  # for displaying images
from utils.downloads import gdrive_download  # for downloading models/datasets
print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 1.12.1+cu113 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [5]:
#follow the link below to get your download code from from Roboflow
!pip install -q roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="roboflow-yolov5")

     |████████████████████████████████| 54 kB 1.9 MB/s 
     |████████████████████████████████| 138 kB 42.4 MB/s 
     |████████████████████████████████| 67 kB 6.0 MB/s 
     |████████████████████████████████| 178 kB 61.5 MB/s 
     |████████████████████████████████| 145 kB 70.1 MB/s 
     |████████████████████████████████| 1.1 MB 58.6 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=roboflow-yolov5


# Let's see the data.yml content

In [49]:
base_path + 'data.yaml'

'/content/drive/MyDrive/PCB_Defect_Detection/data.yaml'

In [50]:
%cat base_path + 'data.yaml'

cat: base_path: No such file or directory
cat: +: No such file or directory
names:
- missing_hole
- mouse_bite
- open_circuit
- short
- spur
- spurious_copper
nc: 6
train: /content/drive/MyDrive/PCB_Defect_Detection/train/images
val: /content/drive/MyDrive/PCB_Defect_Detection/val/images


# Lets get the number of class from data.yml file

In [3]:
data_yml_path = base_path + 'data.yaml'


# define number of classes based on YAML
import yaml
with open(data_yml_path, 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [4]:
num_classes

'6'

# Let's write the number of class in `yolo small` model `yaml file`

In [11]:
#this is the model configuration we will use for our tutorial 
%cat base_path + 'yolov5/models/yolov5s.yaml'

cat: base_path: No such file or directory
cat: +: No such file or directory
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],

# This helps to write the number of classes to custom yaml file

In [12]:
#customize iPython writefile so we can write variables

from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [27]:
# location to save custom_yolov5s.yaml

%%writetemplate /content/drive/MyDrive/PCB_Defect_Detection/yolov5/models/custom_yolov5s.yaml



# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

# Train Custom YOLOv5 Detector

Next, we'll fire off training!:

Here, we are able to pass a number of arguments:


1.   `img:` define input image size
2.   `batch:` determine batch size
1.   `epochs:` define the number of training epochs. (Note: often, 3000+ are common here!)
2.   `data:` set the path to our yaml file
1.   `cfg:` specify our model configuration
2.   `weights:` specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive folder)
1.   `name:` result names
2.   `batch:` determine batch size
1.   `nosave:` only save the final checkpoint
2.   `cache:` cache images for faster training



In [ ]:
# time its performance
%%time

%cd /content/drive/MyDrive/PCB_Defect_Detection/yolov5

!python train.py --img 416 --batch 16 --epochs 3 --data '/content/drive/MyDrive/PCB_Defect_Detection/data.yaml' --cfg '/content/drive/MyDrive/PCB_Defect_Detection/yolov5/models/custom_yolov5s.yaml' --weights '' --name yolov5s_results  --cache

/content/drive/MyDrive/PCB_Defect_Detection/yolov5
train: weights=, cfg=/content/drive/MyDrive/PCB_Defect_Detection/yolov5/models/custom_yolov5s.yaml, data=/content/drive/MyDrive/PCB_Defect_Detection/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 291 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.7.14 torch-1.12.1+cu113 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.93

# Evaluate Custom YOLOv5 Detector Performance

Training losses and performance metrics are saved to Tensorboard and also to a logfile defined above with the --name flag when we train. In our case, we named this yolov5s_results. (If given no name, it defaults to results.txt.) The results file is plotted as a png after training completes.


Note from Glenn: Partially completed results.txt files can be plotted with from utils.utils import plot_results; plot_results().

In [39]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

<IPython.core.display.Javascript object>

In [45]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason... 
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/drive/MyDrive/PCB_Defect_Detection/yolov5/runs/train/yolov5s_results/results.png', width=1000)  # view results.png


TypeError: ignored

# Curious? Visualize Our Training Data with Labels

After training starts, view train*.jpg images to see training images, labels and augmentation effects.

Note a mosaic dataloader is used for training (shown below), a new dataloading concept developed by Glenn Jocher and first featured in YOLOv4.

In [ ]:
# first, display our ground truth data
print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results/val_batch0_labels.jpg', width=900)

In [ ]:
# print out an augmented training example

print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900)

# Run Inference With Trained Weights

Run inference with a pretrained checkpoint on contents of test/images folder

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/

In [ ]:
%ls runs/train/yolov5s_results/weights

In [ ]:
# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
# use the best weights!
%cd /content/yolov5/
python detect.py --weights D:/Ineuron/Project_workshop/pcb_defect_detection/Weights/best.pt --img 416 --conf 0.8 --source D:/Ineuron/Project_workshop/pcb_defect_detection/delete/train_val_test/train_val_test/val/images

In [ ]:
#display inference on ALL test images
#this looks much better with longer training above

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp2*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# Export Trained Weights for Future Inference

Now that you have trained your custom detector, you can export the trained weights you have made here for inference on your device elsewhere

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cp /content/yolov5/runs/train/yolov5s_results/weights/best.pt /content/gdrive/MyDrive/Research/SignLanguageDetection

# Reference blogs

- https://towardsdatascience.com/how-to-train-a-custom-object-detection-model-with-yolo-v5-917e9ce13208

- https://github.com/entbappy/Sign-Language-Generation-From-Video-using-YOLOV5

